<a href="https://colab.research.google.com/github/srgari/Jupyter-Notebook-Labs/blob/master/Copy_of_4%C2%BA_EABDA_Demo_Descomplicando_Modelos_BERT_na_Teoria_e_na_Pr%C3%A1tica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalando requerimentos**

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.1MB 3.3MB/s 
     |████████████████████████████████| 890kB 16.0MB/s 
     |████████████████████████████████| 3.0MB 24.0MB/s 
     |████████████████████████████████| 1.1MB 40.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=7792cdd868481cb909091ebd5bcfdc0c6a87de8c7fba55c56ff0f261b3bddc80
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# **Baixando dados e scripts**

In [ ]:
# Gdrive URL: https://drive.google.com/file/d/1hFktphjfqzAJyeaUJ0dU4DcN4vEFWo8g/view?usp=sharing
! gdown --id 1hFktphjfqzAJyeaUJ0dU4DcN4vEFWo8g -O BERT_notebook.zip
! unzip BERT_notebook.zip

Downloading...
From: https://drive.google.com/uc?id=1hFktphjfqzAJyeaUJ0dU4DcN4vEFWo8g
To: /content/BERT_notebook.zip
5.44MB [00:00, 85.3MB/s]
Archive:  BERT_notebook.zip
   creating: data/
  inflating: data/dataset_real.tsv   
  inflating: data/dataset_toy.tsv    
   creating: scripts/
   creating: scripts/.ipynb_checkpoints/
  inflating: scripts/.ipynb_checkpoints/BERT_Prаtica-checkpoint.ipynb  
  inflating: scripts/BERT_Prаtica.ipynb  
   creating: scripts/images/
  inflating: scripts/images/model_pieces.png  
  inflating: scripts/images/output_pieces.png  
  inflating: scripts/images/pooled_output.png  
  inflating: scripts/images/tokenizer_outputs.png  
  inflating: scripts/images/token_encoder.png  
  inflating: scripts/images/word_outputs.png  


In [ ]:
# Entrar na pasta scripts
import os
os.chdir('scripts/')

# ignore future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# **Importações**

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
from IPython.display import Image

# **Carregando o modelo e tokenizador a serem utilizados**

In [ ]:
#Carregue o modelo e tokenizador a serem usados:
pre_trained_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pre_trained_model_name)
model = BertModel.from_pretrained(pre_trained_model_name)

In [ ]:
#Vamos visualizar!
img = Image(filename = "images/token_encoder.png", width=400)
display(img)

# **Criando a classe de armazenamento de dados**

In [ ]:
#Classe de manipulação de dataset:
class Dataset(Dataset):

    #Construtor da classe:
    def __init__(self, sentences, labels, tokenizer, max_length):
        #Armazene as entradas que serão passadas ao modelo:
        self.sentences = sentences
        
        #Armazene as labels que serão utilizadas para treino/validação/teste:
        self.labels = labels
        
        #Armazene o tokenizador:
        self.tokenizer = tokenizer
        
        #Armazene o tamanho máximo das sentenças:
        self.max_length = max_length
    
    #Retorna o número de instâncias:
    def __len__(self):
        return len(self.sentences)
    
    #Retorna uma instância completa com base num índice:
    def __getitem__(self, index):
        #Obtenha a entrada do índice pertinente:
        sentence = self.sentences[index]
        
        #Obtenha a label do índice pertinente:
        label = self.labels[index]
        
        #Tokenize a entrada:
        encoding = self.tokenizer.encode_plus(
          sentence,
          add_special_tokens=True,
          max_length=self.max_length,
          return_token_type_ids=True,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt',
          truncation=True
        )
    
        #Obtenha os códigos numéricos da sentença:
        input_ids = encoding['input_ids'].flatten()
        
        #Obtenha os códigos numéricos dos token types:
        token_type_ids = encoding['token_type_ids'].flatten()
        
        #Obtenha a máscara de atenção da sentença:
        attention_mask = encoding['attention_mask'].flatten()
        
        #Transforme a label da instância em um tensor:
        label_tensor = torch.tensor(label, dtype=torch.long)
        
        #Retorne um dicionário com estes dados:
        return {
          'input_ids': input_ids,
          'token_type_ids': token_type_ids,
          'attention_mask': attention_mask,
          'labels': label_tensor
        }

In [ ]:
#Vamos visualizar!
img = Image(filename = "images/tokenizer_outputs.png", width=600)
display(img)

# **Criando o carregador de dados**

In [ ]:
#Vamos testar o dataset:
df = pd.read_csv('../data/dataset_toy.tsv', sep='\t')
print(df)

In [ ]:
#Vamos pegar sentenças e etiquetas do dataset:
sentences = df['sentence'].values
labels = df['label'].values

print("Primeira sentença:", sentences[0])
print("Primeira etiqueta:", labels[0])

In [ ]:
#Crie um dataset de testes:
dtoy = Dataset(sentences, labels, tokenizer, max_length=45)

#Pegue uma instância do dataset:
data_inst = next(iter(dtoy))

#Imprima os componentes da instância:
print("Input IDs:", data_inst['input_ids'])
print("Token Type IDs:", data_inst['token_type_ids'])
print("Attention Mask:", data_inst['attention_mask'])
print("Labels:", data_inst['labels'])

In [ ]:
#Crie um DataLoader e coloque o dataset dentro:
dltoy = DataLoader(dtoy, batch_size=3)

#Pegue uma nova batch do DataLoader:
batch = next(iter(dltoy))

#Imprima os componentes da batch:
print("Input IDs:", batch['input_ids'])
print("Token Type IDs:", batch['token_type_ids'])
print("Attention Mask:", batch['attention_mask'])
print("Labels:", batch['labels'])

#**Criando o modelo**

In [ ]:
#Classificador de sentimento:
class SentimentClassifier(torch.nn.Module):

    #Construtor da classe
    def __init__(self, n_classes, pre_trained_model_name):
    
        #Inicialize o modelo:
        super(SentimentClassifier, self).__init__()
        
        #Carregue um modelo BERT pré-treinado:
        self.bert = BertModel.from_pretrained(pre_trained_model_name)
        
        #Crie a camada linear final para classificação:
        self.linear = torch.nn.Linear(self.bert.config.hidden_size, n_classes)

    #Função de execução do modelo:
    def forward(self, input_ids, attention_mask):
        #Passe a entrada pelo modelo BERT:
        word_outputs, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        
        #Passe a saída "pooled" do modelo BERT à camada linear:
        return self.linear(pooled_output)

In [ ]:
#Vamos visualizar!
img = Image(filename = "images/model_pieces.png", width=1000)
display(img)

In [ ]:
#Vamos visualizar!
img = Image(filename = "images/output_pieces.png", width=1000)
display(img)

In [ ]:
#Vamos visualizar!
img = Image(filename = "images/word_outputs.png", width=1000)
display(img)

In [ ]:
#Vamos visualizar!
img = Image(filename = "images/pooled_output.png", width=1000)
display(img)



```
# Isto está formatado como código
```

# **Treinando o modelo**

In [ ]:
def train_model(model, data_loader, loss_fn, optimizer, scheduler):
    #Coloque o modelo em modo de treinamento:
    model = model.train()
    
    #Inicialize o erro total da epoch:
    total_loss = 0
    total_preds = []
    
    #Para cada batch do data_loader, faça:
    for d in data_loader:
        #Obtenha os dados da batch:
        input_ids = d["input_ids"]
        attention_mask = d["attention_mask"]
        labels = d["labels"]
        
        #Passe os dados pelo modelo:
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        #Obtenha as predições:
        _, preds = torch.max(outputs, dim=1)
        total_preds.extend([p.item() for p in preds])
        
        #Calcule o erro:
        loss = loss_fn(outputs, labels)
        total_loss += loss.item()
        
        #Propague o erro para o modelo, promovendo aprendizado:
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    return total_preds, total_loss

In [ ]:
def test_model(model, data_loader, loss_fn):
    #Coloque o modelo em modo de treinamento:
    model = model.eval()
    
    #Inicialize o erro total da epoch:
    total_loss = 0
    total_preds = []
    
    #Para cada batch do data_loader, faça:
    for d in data_loader:
        #Obtenha os dados da batch:
        input_ids = d["input_ids"]
        attention_mask = d["attention_mask"]
        labels = d["labels"]
        
        #Passe os dados pelo modelo:
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        #Obtenha as predições:
        _, preds = torch.max(outputs, dim=1)
        total_preds.extend([p.item() for p in preds])
        
        #Calcule o erro:
        loss = loss_fn(outputs, labels)
        total_loss += loss.item()
    return total_preds, total_loss

In [ ]:
#Carregue os dados:
df = pd.read_csv('../data/dataset_real.tsv', sep='\t')
sentences = df['sentence'].values[:100]
labels = df['label'].values[:100]

#Defina as proporções de treinamento/validação/teste:
tr_prop = 0.9
va_prop = 0.05
te_prop = 0.05

#Obtenha o tamanho do Dataset:
dataset_size = len(sentences)

#Calcules os índices de divisão de treinamento/validação/teste:
tr_limit = int(tr_prop*dataset_size)
va_limit = int((tr_prop+va_prop)*dataset_size)
te_limit = int((tr_prop+va_prop+te_prop)*dataset_size)

#Divida-os para treinamento/validação/teste:
sentences_tr = sentences[:tr_limit]
labels_tr = labels[:tr_limit]
sentences_va = sentences[tr_limit:va_limit]
labels_va = labels[tr_limit:va_limit]
sentences_te = sentences[va_limit:te_limit]
labels_te = labels[va_limit:te_limit]

In [ ]:
#Estabeleça parâmetros para o Dataset:
max_length = 50
batch_size = 32
num_workers = 0

#Crie Datasets para os dados:
dtr = Dataset(sentences_tr, labels_tr, tokenizer, max_length)
dva = Dataset(sentences_va, labels_va, tokenizer, max_length)
dte = Dataset(sentences_te, labels_te, tokenizer, max_length)

#Crie DataLoaders para os Datasets:
dltr = DataLoader(dtr, batch_size=batch_size, num_workers=num_workers)
dlva = DataLoader(dva, batch_size=batch_size, num_workers=num_workers)
dlte = DataLoader(dte, batch_size=batch_size, num_workers=num_workers)

In [ ]:
#Crie o modelo:
sentiment_classifier = SentimentClassifier(n_classes=2, pre_trained_model_name=pre_trained_model_name)

#Crie elementos de treinamento:
epochs = 3
total_steps = len(dltr)*epochs
loss_function = torch.nn.CrossEntropyLoss()
optimizer = AdamW(sentiment_classifier.parameters())
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [ ]:
#Treine o modelo:
for i in range(epochs):
    #Treine em dados de treinamento:
    print('\nTreinando o modelo, epoch ', i)
    preds_tr, total_loss_tr = train_model(sentiment_classifier, dltr, loss_function, optimizer, scheduler)
    
    #Valide em dados de validação:
    print('Validando o modelo, epoch ', i)
    preds_va, total_loss_va = test_model(sentiment_classifier, dlva, loss_function)
    
    #Imprima os erros de treinamento/validação:
    print('Erro de treinamento:', total_loss_tr)
    print('Erro de validação:', total_loss_va)

# **Testando o modelo**

In [ ]:
#Teste o modelo:
preds_te, total_loss_te = test_model(sentiment_classifier, dlte, loss_function)
print('Labels reais: ', labels_te)
print('Labels preditas: ', preds_te)

#Calcule o erro com as etiquetas reais:
f1 = f1_score(labels_te, preds_te, average='micro')
print('F1-score (micro):', f1)